<a href="https://colab.research.google.com/github/plizeeee/NLP-Amazon-Reviews-Project/blob/main/LSTM_Code_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gzip
import json
from datetime import datetime
import re
import json
import pickle
import random
import numpy as np
import pandas as pd
from html import unescape
from  matplotlib import pyplot as plt
from collections import Counter
from sklearn.manifold import TSNE
from scipy.spatial.distance import cosine
from sklearn.feature_extraction import _stop_words
from collections import defaultdict, Counter
import pickle
import gensim
from gensim.models import FastText
from gensim.test.utils import common_texts  # some example sentences
from sklearn.model_selection import train_test_split
import operator
from random import shuffle
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from datetime import datetime





pd.set_option('display.max_colwidth', -1)
pd.options.mode.chained_assignment = None

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
!nvidia-smi

Wed Apr 28 18:23:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    50W / 250W |   8831MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Loading data from pickle files
!pip3 install pickle5
import pickle5 as pickle

# with open(r'/content/drive/My Drive/df_electronics_2014_all.p', 'rb') as handle:
#     df_electronics_2014 = pickle.load(handle)

# with open('/content/drive/My Drive/df_movietv_2014_all.p', 'rb') as handle:
#     df_movietv_2014 = pickle.load(handle)
    
# with open('/content/drive/My Drive/df_foods_2014_all.p', 'rb') as handle:
#     df_foods_2014 = pickle.load(handle)

start = datetime.now()
with open('/content/drive/My Drive/df_train.p', 'rb') as handle:
    df_train = pickle.load(handle)
    
with open('/content/drive/My Drive/df_dev.p', 'rb') as handle:
    df_dev = pickle.load(handle)
    
with open('/content/drive/My Drive/df_test.p', 'rb') as handle:
    df_test = pickle.load(handle)
    
    
print(datetime.now()-start)

     |████████████████████████████████| 133kB 8.0MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219250 sha256=e88d85858639441319662264802cce3e4e57a13a3cd8e26a8322d6854e1bf5f8
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5
0:00:10.883530


In [ ]:
# Loading fasttext emdeddings 

# with open('fast_text_vecs_wikinews.p', 'wb') as handle:
#     pickle.dump(fast_text_vecs, handle, protocol=pickle.HIGHEST_PROTOCOL)

start = datetime.now()
with open('/content/drive/My Drive/fast_text_vecs_wikinews.p', 'rb') as handle:
    fast_text_vecs = pickle.load(handle)
print(datetime.now()-start)

0:00:35.293977


In [ ]:
# Create dictionary for word look-up
word_counter = Counter([w for a in df_train['cleaned_text_lst'] for w in a if w in fast_text_vecs.keys()])
w2id = {w: i + 2 for i, w in enumerate(w for w, c in word_counter.most_common())}

# Create dictionary for reverse word look-up
id2w = {i: w for w, i in w2id.items()}

In [ ]:
# Define dataset class
class LSTMDataset(Dataset):

    def __init__(self, data, w2id):
        
        # Encode abstracts
        self.review = list(data['cleaned_text_lst'].apply(lambda x: [w2id[w] if w in w2id else 1 for w in x]))

        self.summary = list(data['summary_cleaned'].apply(lambda x: [w2id[w] if w in w2id else 1 for w in x]))
        
        # Store labels
        self.labels = list(data['class'])

        self.years = list(data['review_year_normalized'])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        review = self.review[idx]
        label = self.labels[idx]
        summary = self.summary[idx]
        years = self.years[idx]
        return review, label, summary, years

In [ ]:
# Define collate function
def lstm_collate(batch):
    
    # Store batch size
    batch_size = len(batch)

    # Separate abstracts and labels
    reviews = [a for a, _, _, _ in batch]
    summaries = [s for _, _, s, _ in batch]
    labels = torch.tensor([l for _, l, _, _ in batch]).long()
    years = torch.tensor([y for _, _ , _, y in batch]).float()

    # Store length of longest abstract in batch
    max_len = max(len(a) for a in reviews)

    # Create padded review tensors
    reviews_pad = torch.zeros((batch_size, max_len)).long()
    for i, a in enumerate(reviews):
        reviews_pad[i, :len(a)] = torch.tensor(a)


    # Store length of longest summaries in batch
    max_len_summaries = max(len(a) for a in summaries)

    # Create padded summaries tensors
    summaries_pad = torch.zeros((batch_size, max_len_summaries)).long()
    for i, a in enumerate(summaries):
        summaries_pad[i, :len(a)] = torch.tensor(a)

    return reviews_pad, labels, summaries_pad, years

In [ ]:
# Define LSTM classifier
class LSTMClassifier(torch.nn.Module):
    
    def __init__(self, pretrained_emb, hidden_dim, output_dim,dropout_val,bidirectional_bool):
        
        # Define network layers
        super(LSTMClassifier, self).__init__()
        self.embedding = torch.nn.Embedding.from_pretrained(pretrained_emb, padding_idx=0)
        self.lstm = torch.nn.LSTM(pretrained_emb.shape[1], hidden_dim, batch_first=True,bidirectional = bidirectional_bool)

        if bidirectional_bool:
          self.linear = torch.nn.Linear(2*hidden_dim, output_dim)
        else:
          self.linear = torch.nn.Linear(hidden_dim, output_dim)
        
        # Define dropout
        self.dropout = torch.nn.Dropout(dropout_val)
        
    def forward(self, reviews):
            
        # Define flow of tensors through network
        emb = self.embedding(reviews)
        output, (hidden, cell) = self.lstm(self.dropout(emb))
        # print(output.shape)
        return self.linear(self.dropout(output[:, -1, :]))

In [ ]:
# Combinations of features, models and categories used for model training in the loop of the next cell
categories = ['Moviestv','Books']
bidirectional = [True,False]
feature_lst = ['content_feats','summary_feats']

category = 'Moviestv'
df_train_category = df_train[df_train['category'] == category].copy()
df_dev_category = df_dev[df_dev['category'] == category].copy()
df_test_category = df_test[df_test['category'] == category].copy()

# Create matrix of pretrained embeddings
fasttext_emb = torch.tensor([fast_text_vecs[id2w[i]] for i in range(2, len(id2w) + 2) if id2w[i] in fast_text_vecs.keys()]).float()
fasttext_emb = torch.cat((torch.zeros((1, 300)), fasttext_emb.mean(axis=0, keepdim=True), fasttext_emb), axis=0)


In [ ]:
# bidirectional_bool = False

# if category
# Create datasets

category_accs_dict = dict()
category_cm_dict = dict()

start_time = datetime.now()

for category in categories:
  df_train_category = df_train[df_train['category'] == category].copy()
  df_dev_category = df_dev[df_dev['category'] == category].copy()
  df_test_category = df_test[df_test['category'] == category].copy()


  train_dataset = LSTMDataset(df_train_category, w2id)
  dev_dataset = LSTMDataset(df_dev_category, w2id)
  test_dataset = LSTMDataset(df_test_category, w2id)

  # Create data loaders
  train_loader = DataLoader(train_dataset, batch_size=250, collate_fn=lstm_collate, shuffle=True)
  dev_loader = DataLoader(dev_dataset, batch_size=250, collate_fn=lstm_collate)
  test_loader = DataLoader(test_dataset, batch_size=250, collate_fn=lstm_collate)


  bid_accs_dict = dict()
  bid_cm_dict = dict()

  for bidirectional_bool in bidirectional:


    accs = dict()
    cm = dict()
    for feature_type in feature_lst:
      val_acc = 0.0
      while val_acc < 0.25: # Might need to repeat training, since occasionally the LSTM cannot properly navigate the loss space and gets stuck.
        print('\n\n-------------------------------------------------------------------------------')
        print('category = ',category, '       bidirectional_bool = ',bidirectional_bool, '        feature type = ',feature_type)


        if feature_type =='content_feats':
          print('\n\n\n\n')
          # Initialize model
          model_content = LSTMClassifier(fasttext_emb, 200, 6,0.25,bidirectional_bool)

          # Define optimizer and training objective
          optimizer = torch.optim.Adam(model_content.parameters(), lr=0.002)
          criterion = torch.nn.CrossEntropyLoss()

          # Define device and move model to CUDA if available
          device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
          model_content = model_content.to(device)

          # Train model


          min_epochs = 35
          min_val_loss = 100 # set to an arbitrarily high number for starting out
          num_no_progress = 0
          early_stopping_num_epochs = 7

          for e in range(1, 81):
              model_content.train()

              for i, b in enumerate(train_loader):

                  # Perform forward pass
                  optimizer.zero_grad()
                  reviews, lbls, summaries, years = [t.to(device) for t in b]
          #         print(reviews.max())
                  output = model_content(reviews)
                  loss = criterion(output, lbls)

                  # Perform backpropagation and update weights
                  loss.backward()
                  optimizer.step()
                  if i%100==0:
                      print('batch num:', i)

              # Evaluate model on development data
              model_content.eval()

              y_true = list()
              y_pred = list()

              with torch.no_grad():
                  for i, b in enumerate(dev_loader):
                      reviews, lbls, summaries, years = [t.to(device) for t in b]
                      output = model_content(reviews)
                      max_output = output.argmax(dim=1)
                      y_true.extend(lbls.tolist())
                      y_pred.extend(max_output.tolist())

                  # Computations for early stopping
                  # loss = criterion(output_all, lbls_all)


                      if i == 0:
                        output_tot = output 
                        lbls_all = lbls
                      else:
                        output_tot = torch.cat((output_tot, output), 0)
                        lbls_all = torch.cat((lbls_all, lbls), 0)
                  loss_val = criterion(output_tot, lbls_all)  


                  val_acc = accuracy_score(y_true, y_pred)
                  if loss_val < min_val_loss:
                      min_val_loss = loss_val
                      num_no_progress = 0
                  else:
                      num_no_progress += 1
                      
                  if num_no_progress >= early_stopping_num_epochs and e > min_epochs and val_acc > 0.35:
                      break

                print('Development accuracy after {} epoch(s): {:.2f}      val loss: {},     num epochs no progress: {}'.format(e, val_acc,loss_val.item(),num_no_progress))

        else:

          print('\n\n\n\n')
          # Initialize model
          model_summaries = LSTMClassifier(fasttext_emb, 200, 6,0.25,bidirectional_bool)

          # Define optimizer and training objective
          optimizer = torch.optim.Adam(model_summaries.parameters(), lr=0.002)
          criterion = torch.nn.CrossEntropyLoss()

          # Define device and move model to CUDA if available
          device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
          model_summaries = model_summaries.to(device)

          # Train model

          min_epochs = 25
          min_val_loss = 100 # set to an arbitrarily high number for starting out
          num_no_progress = 0
          early_stopping_num_epochs = 5

          for e in range(1, 51):
              model_summaries.train()

              for i, b in enumerate(train_loader):

                  # Perform forward pass
                  optimizer.zero_grad()
                  reviews, lbls, summaries, years = [t.to(device) for t in b]
          #         print(reviews.max())
                  output = model_summaries(summaries)
                  loss = criterion(output, lbls)

                  # Perform backpropagation and update weights
                  loss.backward()
                  optimizer.step()
                  if i%100==0:
                      print('batch num:', i)

              # Evaluate model on development data
              model_summaries.eval()

              y_true = list()
              y_pred = list()

              with torch.no_grad():
                  for i, b in enumerate(dev_loader):
                      reviews, lbls, summaries, years = [t.to(device) for t in b]
                      output = model_summaries(summaries)
                      max_output = output.argmax(dim=1)
                      y_true.extend(lbls.tolist())
                      y_pred.extend(max_output.tolist())

                  # Computations for early stopping
                  # loss = criterion(output_all, lbls_all)


                      if i == 0:
                        output_tot = output 
                        lbls_all = lbls
                      else:
                        output_tot = torch.cat((output_tot, output), 0)
                        lbls_all = torch.cat((lbls_all, lbls), 0)
                  loss_val = criterion(output_tot, lbls_all)  


                  val_acc = accuracy_score(y_true, y_pred)
                  if loss_val < min_val_loss:
                      min_val_loss = loss_val
                      num_no_progress = 0
                  else:
                      num_no_progress += 1
                      
                  if num_no_progress >= early_stopping_num_epochs and e > min_epochs and val_acc > 0.25:
                      break

              print('Development accuracy after {} epoch(s): {:.2f}      val loss: {},     num epochs no progress: {}'.format(e, val_acc,loss_val.item(),num_no_progress))

    # Once models are trained build the ensemble and record the accuracies. Also save the test accuracy of the "content" model.
    softmax_outputs = torch.nn.Softmax(dim=1)
    y_true = list()
    y_pred_ensemble = list()
    y_pred_summaries = list()
    y_pred_content = list()
    nb_classes = 6
    confusion_matrix_ensemble = torch.zeros(nb_classes, nb_classes)
    confusion_matrix_summaries = torch.zeros(nb_classes, nb_classes)
    confusion_matrix_content = torch.zeros(nb_classes, nb_classes)

    with torch.no_grad():
        for i, b in enumerate(test_loader):
            reviews, lbls, summaries, years = [t.to(device) for t in b]
            output_content = softmax_outputs(model_content(reviews))
            output_summaries = softmax_outputs(model_summaries(summaries))
            
            # Adding the probabilities of the classes together (I tried weighing each term based on the accuracy, but does not appear to work well)
            output_tot = output_content + output_summaries

            max_output_tot = output_tot.argmax(dim=1)
            y_true.extend(lbls.tolist())
            y_pred_ensemble.extend(max_output_tot.tolist())

            max_output_content = output_content.argmax(dim=1)
            y_pred_content.extend(max_output_content.tolist())

            max_output_summaries = output_summaries.argmax(dim=1)
            y_pred_summaries.extend(max_output_summaries.tolist())

            for t, p in zip(lbls.view(-1), max_output_tot.view(-1)):
                confusion_matrix_ensemble[t.long(), p.long()] += 1

            for t, p in zip(lbls.view(-1), max_output_content.view(-1)):
                confusion_matrix_content[t.long(), p.long()] += 1
            
            for t, p in zip(lbls.view(-1), max_output_summaries.view(-1)):
                confusion_matrix_summaries[t.long(), p.long()] += 1
            



    test_acc_content = accuracy_score(y_true, y_pred_content)
    print('Content only Accuracy')
    print(test_acc_content)
    print(confusion_matrix_content/1000)
    accs['content'] = test_acc_content
    cm['content'] = confusion_matrix_content/1000

    test_acc_summaries = accuracy_score(y_true, y_pred_summaries)
    print('Summaries only Accuracy')
    print(test_acc_summaries)
    print(confusion_matrix_summaries/1000)
    accs['summaries'] = test_acc_summaries
    cm['summaries'] = confusion_matrix_summaries/1000


    test_acc_ensemble = accuracy_score(y_true, y_pred_ensemble)
    print('Ensemble Accuracy')
    print(test_acc_ensemble)
    print(confusion_matrix_ensemble/1000)
    accs['ensemble'] = test_acc_ensemble
    cm['ensemble'] = confusion_matrix_ensemble/1000


    bid_accs_dict[bidirectional_bool] = accs
    bid_cm_dict[bidirectional_bool] = cm

  category_accs_dict[category] = bid_accs_dict
  category_cm_dict[category] = bid_cm_dict

print(datetime.now()-start_time)



-------------------------------------------------------------------------------
category =  Moviestv        bidirectional_bool =  True         feature type =  content_feats





batch num: 0
batch num: 100
Development accuracy after 1 epoch(s): 0.17      val loss: 1.7920277118682861,     num epochs no progress: 0
batch num: 0
batch num: 100
Development accuracy after 2 epoch(s): 0.17      val loss: 1.7918795347213745,     num epochs no progress: 0
batch num: 0
batch num: 100
Development accuracy after 3 epoch(s): 0.17      val loss: 1.7917841672897339,     num epochs no progress: 0
batch num: 0
batch num: 100
Development accuracy after 4 epoch(s): 0.17      val loss: 1.7919726371765137,     num epochs no progress: 1
batch num: 0
batch num: 100
Development accuracy after 5 epoch(s): 0.17      val loss: 1.7920682430267334,     num epochs no progress: 2
batch num: 0
batch num: 100
Development accuracy after 6 epoch(s): 0.17      val loss: 1.7919749021530151,     num epochs no progress: 

In [ ]:
# category_accs_dict
category_cm_dict['Books'][True]['ensemble']

# Accuracy breakdown for each model
rating_types = ['negative rating','medium rating','positive rating']
for category in category_cm_dict.keys():
  for bidirectional_bool in category_cm_dict[category].keys():
    for model_type in category_cm_dict[category][bidirectional_bool].keys():
      print('overall acc:',category_accs_dict[category][bidirectional_bool][model_type])
      print(category,'   bidirectional:', bidirectional_bool,'    model type:', model_type)
      print(category_cm_dict[category][bidirectional_bool][model_type])
      per_class_accs = category_cm_dict[category][bidirectional_bool][model_type].diag()/category_cm_dict[category][bidirectional_bool][model_type].sum(1)
      avg_acc_per_sentiment = {rating_types[i]:((per_class_accs[2*i]+per_class_accs[2*i+1])/2).tolist() for i in range(3)}
      print(per_class_accs)
      print('Average accuracy per review helpfulness category')
      print(avg_acc_per_sentiment)
      print('\n\n')

overall acc: 0.4633333333333333
Moviestv    bidirectional: True     model type: content
tensor([[0.5430, 0.1350, 0.1780, 0.0530, 0.0840, 0.0070],
        [0.2180, 0.4320, 0.1120, 0.1720, 0.0480, 0.0180],
        [0.1640, 0.0780, 0.3660, 0.1850, 0.1430, 0.0640],
        [0.0260, 0.1070, 0.2060, 0.4420, 0.0680, 0.1510],
        [0.0530, 0.0430, 0.1090, 0.0820, 0.4870, 0.2260],
        [0.0090, 0.0220, 0.0510, 0.1450, 0.2630, 0.5100]])
tensor([0.5430, 0.4320, 0.3660, 0.4420, 0.4870, 0.5100])
Average accuracy per review helpfulness category
{'negative rating': 0.48750001192092896, 'medium rating': 0.4039999842643738, 'positive rating': 0.4984999895095825}



overall acc: 0.3605
Moviestv    bidirectional: True     model type: summaries
tensor([[0.4240, 0.1510, 0.1780, 0.0850, 0.1180, 0.0440],
        [0.2680, 0.3230, 0.1680, 0.1180, 0.0760, 0.0470],
        [0.1740, 0.0770, 0.3180, 0.1970, 0.1220, 0.1120],
        [0.0880, 0.1100, 0.2270, 0.2990, 0.1120, 0.1640],
        [0.0900, 0.0530, 0.

In [ ]:
# Saving results to a pickle file to be used outside of colab
with open('lstm_accuracies.pickle', 'wb') as handle:
    pickle.dump(category_accs_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('lstm_confusion_mats.pickle', 'wb') as handle:
    pickle.dump(category_cm_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
